In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.utils import shuffle
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.svm import SVC

In [ ]:
pd.set_option('display.expand_frame_repr', False)
spoty_df = pd.read_csv('SpotifyFeatures.csv')

spoty_df = shuffle(spoty_df)

### Dataset structure

In [ ]:
print(spoty_df.head())
print(spoty_df.describe())
print(spoty_df.info())
print(spoty_df['genre'])

### management of null, infinite and duplicate values

In [ ]:
#checking null values
null_values = spoty_df.isnull()
null_columns = null_values.any()
print("Colonne con valori nulli:")
print(null_columns)

In [ ]:
#checking infinite values
if spoty_df.select_dtypes(include=[np.number]).applymap(np.isinf).any().any():
    print("Sono presenti valori infiniti nel DataFrame.")
else:
    print("Non sono presenti valori infiniti nel DataFrame.")

In [ ]:
#checking and removing duplicates
duplicates = spoty_df[spoty_df.duplicated(subset=['track_id'])]
print("duplicates:")
print(duplicates)
spoty_df = spoty_df.drop_duplicates(subset=['track_id'])
genre_counts = spoty_df['genre'].value_counts()
print(genre_counts)

### Selection of genres and elimination of uninteresting features

In [ ]:
#Selection of the genres of interest.
desired_genres = ['Rock', 'Classical', 'Jazz']
top3_df = spoty_df[spoty_df['genre'].isin(desired_genres)]
genre_counts = top3_df['genre'].value_counts()
print(genre_counts)

In [ ]:
#Elimination of features that are certainly useless.
top3_df = top3_df.drop(['artist_name'], axis=1)
top3_df = top3_df.drop(['track_name'], axis=1)
top3_df = top3_df.drop(['track_id'], axis=1)
top3_df = top3_df.drop(['duration_ms'], axis=1)

# EDA

In [ ]:
# Selection of only the numerical columns.
features = top3_df.select_dtypes(include=np.number).columns.tolist()

In [ ]:
grouped = top3_df.groupby(['genre', 'key']).size().unstack(fill_value=0)
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.barplot(data=grouped, palette="muted")
plt.xlabel('Valore di key')
plt.ylabel('Numero di campioni')
plt.title('Numero di campioni per genere musicale e valore di key')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Handling of time_signature

In [ ]:
print(top3_df['time_signature'].unique())
#It is very likely that the entry of 1/4 is a data entry error because it is equivalent to 4/4.
top3_df.loc[:, 'time_signature'] = top3_df['time_signature'].replace(['1/4'], '4/4')
print("Tracks with a 0/4 time signature: %s" % len(top3_df[top3_df['time_signature'] == '0/4']))
top3_df = top3_df[top3_df['time_signature'] != '0/4']  # Remove the rows with '0/4' from the 'time_signature' column.

### Genre distribution

In [ ]:
plt.figure(figsize=(15, 5))
sns.histplot(top3_df['genre']).set_title('Genre Distribution')
plt.show()

In [ ]:
# We notice that the dataset is unbalanced.

## Conversion of categorical values

In [ ]:
#We use labels because they are ordinal categorical values
time_signature_mapping = {'3/4': 0,'4/4':1 ,'5/4': 2}
top3_encoded_df=top3_df
top3_encoded_df['time_signature'] = top3_encoded_df['time_signature'].map(time_signature_mapping)
mode_mapping = {'Major': 1,'Minor':0}
top3_encoded_df['mode'] = top3_encoded_df['mode'].map(mode_mapping)
notes_ordered = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
key_mapping = {note: index for index, note in enumerate(notes_ordered)}
top3_encoded_df['key'] = top3_encoded_df['key'].map(key_mapping)
print(top3_df.head())

## Normalization

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

top3_encoded_df['loudness'] *= -1

# Splitting labels from features
t = top3_encoded_df['genre']
X = top3_encoded_df.drop(['genre'], axis=1)

# Splitting the dataset in train and test
X_train, X_test, t_train, t_test = train_test_split(X, t, train_size=0.7, test_size=0.3, random_state=100)

# Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

#Merging X_train and X_test, t_train and t_test into a single DataFrame
X_combined = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)

t_train = pd.DataFrame(t_train).reset_index(drop=True)
t_test = pd.DataFrame(t_test).reset_index(drop=True)

X_train['genre'] = t_train
X_test['genre'] = t_test

df_with_labels = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)

#final dataframe
print("\ncombined_df_with_labels:")
print(df_with_labels)


## Further data exploration

In [ ]:
num_rows = 2
num_cols = (len(features) + 1) // 2
plt.figure(figsize=(16, 10))

for i, feature in enumerate(features, start=1):
    plt.subplot(num_rows, num_cols, i)

    for genre, data in df_with_labels.groupby('genre'):
        counts, bin_edges = np.histogram(data[feature], bins=10, density=True)
        pdf = counts / sum(counts)

        plt.plot(bin_edges[1:], pdf, label=genre)

    plt.title(f'Distribuzione di {feature}')
    plt.xlabel(feature)
    plt.ylabel('Densità di Probabilità')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#We might consider dropping tempo, speechiness, liveness, and instrumentalness since their distributions are largely
#overlapping and thus not sufficiently discriminating.

In [ ]:
#Let's try to understand which features better explain the genres.
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
X_dummies=pd.get_dummies(df_with_labels)
X_dummies.corr()['genre_Rock'].sort_values(ascending=False).plot(kind='bar', ax=axs[0])
axs[0].set_title('Correlazione con il genere Electronic')
X_dummies.corr()['genre_Classical'].sort_values(ascending=False).plot(kind='bar', ax=axs[1])
axs[1].set_title('Correlazione con il genere Classical')
X_dummies.corr()['genre_Jazz'].sort_values(ascending=False).plot(kind='bar', ax=axs[2])
axs[2].set_title('Correlazione con il genere Jazz')
plt.tight_layout()
plt.show()

In [ ]:
#The result of the previous graph is confirmed by this graph. We also notice that key doesn't explain well enough the genre.

In [ ]:
#Analysis of the correlation between features
corr_matrix = df_with_labels.corr(numeric_only=True)
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
#We notice that energy is highly correlated with acousticness, and we might consider dropping one of them.

In [ ]:
df_with_labels=df_with_labels.reset_index(drop=True).drop(columns=['key','instrumentalness','speechiness','liveness','tempo','time_signature'], axis=1)


#top5_encoded_df.drop('energy', axis=1)


# Outlier management

In [ ]:
#grafico box per controllare outlier
plt.figure(figsize=(22, 6))
ax = sns.boxplot(data=df_with_labels.drop(['genre'], axis=1).values)
ax.set_xticklabels(df_with_labels.drop(['genre'], axis=1).columns, rotation=45)  # Imposta i nomi delle colonne come etichette sull'asse x
plt.xlabel('Nome delle feature')  # Aggiungi un'etichetta all'asse x
plt.ylabel('Valore')  # Aggiungi un'etichetta all'asse y
plt.title('Boxplot delle feature')  # Aggiungi un titolo al grafico
plt.show()

#print(X_train.head())
#mi concentro sulle feature 5,6,8 che sono quelle con outlier (tempo, liveness, loudaness e  speechiness)

In [ ]:
#Splitting into train and test to remove outliers only from train
t = df_with_labels['genre']
X = df_with_labels.drop(['genre'], axis=1)

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
selected_columns = X_train[[ 'loudness','popularity']]
selected_data = pd.DataFrame(selected_columns)
plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=selected_data)
plt.xlabel('Features')
plt.ylabel('Valore')
plt.title('Boxplot di alcune features selezionate')
plt.show()

In [ ]:
# Removing outliers from X_train and corresponding records from y_train
quantiles = X_train[['loudness','popularity']].quantile([0.5, 0.95])
lower_quantile = quantiles.iloc[0]
upper_quantile = quantiles.iloc[1]

X_train = X_train[(X_train['loudness'] >= lower_quantile['loudness']) & (X_train['loudness'] <= upper_quantile['loudness'])]
t_train = t_train[X_train.index]  # Elimina i record corrispondenti da t_train

X_train = X_train[(X_train['popularity'] >= lower_quantile['popularity']) & (X_train['popularity'] <= upper_quantile['popularity'])]
t_train = t_train[X_train.index]  # Elimina i record corrispondenti da t_train

# Graph without outliers
selected_columns = X_train[['loudness', 'popularity']]
selected_data = pd.DataFrame(selected_columns)
plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=selected_data)
plt.xlabel('Features')
plt.ylabel('Valore')
plt.title('Boxplot di alcune features selezionate')
plt.show()


## VIF

In [ ]:
vif = pd.DataFrame()
column_names = pd.get_dummies(X_train, drop_first='False').columns.tolist()
vif['Feature'] = column_names
vif['VIF'] = [variance_inflation_factor(X_train, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by="VIF", ascending=False)
print(vif)


In [ ]:
# we notice we could drop energy

In [ ]:
#Let's check if the correlation matrix gives us the same result
corr_matrix = X_train.corr(numeric_only=True)
sns.heatmap(corr_matrix, annot=True)
plt.show()
# we can definetly drop energy

In [ ]:
X_train.drop('energy', axis=1, inplace=True)
X_test.drop('energy', axis=1, inplace=True)
vif = pd.DataFrame()
column_names = pd.get_dummies(X_train, drop_first='False').columns.tolist()
vif['Feature'] = column_names
vif['VIF'] = [variance_inflation_factor(X_train, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by="VIF", ascending=False)
print(vif)

# logistic regression

In [ ]:
hyper_param = [
    {
        'C': np.logspace(-4, 4, 20),
        'solver': ['lbfgs','newton-cg', 'sag', 'saga'],
        'max_iter': [100, 1000, 2500]
    }
]

# Model creation and training using grid search
clf = GridSearchCV(linear_model.LogisticRegression(multi_class='multinomial'), param_grid=hyper_param, verbose=10, n_jobs=-1, error_score="raise", scoring='f1_weighted')
best_clf = clf.fit(X_train, t_train)

# Printing the best parameters and the best accuracy
print("Tuned hyperparameters (best parameters): ", clf.best_params_)
print("F1 score on the train set: ", clf.best_score_)

# Training the best model on the complete training data
best_model = linear_model.LogisticRegression(**clf.best_params_)
best_model.fit(X_train, t_train)

# Prediction on the test set
t_hat_test = best_model.predict(X_test)

# Calculation of evaluation metrics on the test set
print("Accuracy score on the test set: ", accuracy_score(t_test, t_hat_test))
print("F1 score on the test set: ", f1_score(t_test, t_hat_test, average='weighted'))
print(classification_report(t_test, t_hat_test))


## Confusion matrix

In [ ]:
conf_matrix = confusion_matrix(t_test, t_hat_test)

labels = ['Classical', 'Jazz','Rock']
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# KNN

In [ ]:
hyper_param = [
    {
        'n_neighbors': [3, 7,9,11],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
]

clfKNN = GridSearchCV(KNeighborsClassifier(), param_grid=hyper_param, verbose=10, n_jobs=-1, error_score="raise",scoring='f1_macro')
best_clf = clfKNN.fit(X_train, t_train)

# Printing the best parameters and the best accuracy
print("Tuned hyperparameters (best parameters): ", clfKNN.best_params_)
print("Accuracy:", clfKNN.best_score_)

t_hat_test = clfKNN.predict(X_test)
print("Accuracy score on the test set: ", accuracy_score(t_test, t_hat_test))
print("F1 score on the test set: ", f1_score(t_test, t_hat_test, average='weighted'))


# Training the best model on the complete training data
model = KNeighborsClassifier(**clfKNN.best_params_)
model.fit(X_train, t_train)

# Prediction on the test set
t_pred = model.predict(X_test)

# Calculation of evaluation metrics on the test set
print(classification_report(t_test, t_pred))

## ROC-AUC

In [ ]:
# Calculate the class probabilities for the test set
probs = clf.predict_proba(X_test)

# Calculate the ROC-AUC for each class
roc_auc = roc_auc_score(t_test, probs, multi_class='ovr')  # 'ovr' for multiclass problems

#Calculate and plot the ROC curve for each class
plt.figure(figsize=(8, 6))

for i in range(len(labels)):
    fpr, tpr, _ = roc_curve(t_test == labels[i], probs[:, i])
    plt.plot(fpr, tpr, label=f'ROC Curve (Class {labels[i]})')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()

# Neural Network

In [ ]:
print(t_train)

In [ ]:
hyper_param_mlp = {
    'hidden_layer_sizes': [(100,100), (100, 150), (150, 150)],
    'activation': [ 'relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'early_stopping': [True, False],
    'max_iter': [500]
}

clfMLP = GridSearchCV(MLPClassifier(), param_grid=hyper_param_mlp, verbose=10, n_jobs=-1, error_score="raise", scoring='f1_macro')

best_clf_mlp = clfMLP.fit(X_train, t_train)

print("Tuned hyperparameters (best parameters): ", clfMLP.best_params_)
print("Accuracy:", clfMLP.best_score_)

best_mlp_model = MLPClassifier(**clfMLP.best_params_)
best_mlp_model.fit(X_train, t_train)

t_hat_test_mlp = best_mlp_model.predict(X_test)

print("F1 score on the test set: ", f1_score(t_test, t_hat_test_mlp, average='weighted'))

print(classification_report(t_test, t_hat_test_mlp))


# SVM

In [ ]:
hyper_param_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
}

clf_svm = GridSearchCV(SVC(), param_grid=hyper_param_svm, verbose=10, n_jobs=-1, error_score="raise", scoring='f1_macro')

best_clf_svm = clf_svm.fit(X_train, t_train)

print("Tuned hyperparameters (best parameters): ", clf_svm.best_params_)
print("Accuracy:", clf_svm.best_score_)

best_svm_model = SVC(**clf_svm.best_params_)
best_svm_model.fit(X_train, t_train)

t_pred_svm = best_svm_model.predict(X_test)

print("F1 score on the test set: ", f1_score(t_test, t_pred_svm, average='weighted'))
print(classification_report(t_test, t_pred_svm))
